### Interaction with Jupyter Notebook remotely

The notebook does the following

- Connects to user jupyter session using the token
- Queries notebook content in the server
- Queries running kernels
- Executes code on jupyter kernel
- Downloads data 


### Create a new object to communicate with the server

In [18]:
from jupyterhub_client import  RemoteJupyterClient
pp = pprint.PrettyPrinter(indent=4)
import pprint
remote = RemoteJupyterClient('karora','d055cd3b067a455a9fe2432cc313a7f2','rstudio2.luoquant.com/julia')

### Lists down running kernels. At least one kernel should be available to execute the code

In [107]:
remote.kernels()

,id,name,last_activity,execution_state,connections
0,12263c47-20c1-4311-b93a-bf79911e8042,python3,2022-05-19T16:41:06.274127Z,idle,1
1,54ba8661-9f6f-4db5-9cb1-083bada6a501,python3,2022-05-19T17:44:11.140592Z,idle,2


### Read content of a notebook

In [108]:
notebook = remote.content('projects/demo/jupyter-remote/Remote Jupyter Demo.ipynb')

In [112]:
pp.pprint(notebook.loc[0,'source'])

('#Initializes it should be called only once\n'
 'from lquantPy import LQuant\n'
 'wq = LQuant.LQuant()')


In [19]:
pp.pprint(notebook.loc[1,'source'])

('print("Fetching Data....")\n'
 'r1 = '
 "wq.new_request().runFor('SP500').start('2022-01-31').to('2022-02-28').at('1me').a('PRCCD').addInFlag()\n"
 'data1 = wq.get_data(r1)\n'
 'print("Fetching Data Done.")\n'
 'df = data1.as_large_data_frame()\n'
 'print("Exporting to File")\n'
 "df.to_csv('sp500.csv')\n"
 'print("File available for download")')


### Execute Remote Code on the server

In [113]:
response = remote.execute('12263c47-20c1-4311-b93a-bf79911e8042', notebook.loc[0,'source'])

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-12-f316b3568642> in <module>()
      1 #Initializes it should be called only once
      2 from lquantPy import LQuant
----> 3 wq = LQuant.LQuant()

/usr/local/lib/python3.5/dist-packages/lquantPy/LQuant.py in __init__(self)
    179     def __init__(self):
    180 
--> 181         _set_init_parameters()
    182 
    183         self.logger = _get_logger(__name__)

/usr/local/lib/python3.5/dist-packages/lquantPy/LQuant.py in _set_init_parameters()
    101 
    102     import jnius_config
--> 103     jnius_config.add_options('-Xms2g', '-Xmx64g')
    104 
    105     r_home = None

/usr/local/lib/python3.5/dist-packages/jnius_config.py in add_options(*opts)
     24     "Appends options to the list of VM options."
     25     if vm_running:
---> 26         raise ValueError("VM is already running, can't set options")
     27   

In [114]:
response = remote.execute('12263c47-20c1-4311-b93a-bf79911e8042', notebook.loc[1,'source'])

Fetching Data....
Fetching Data Done.
Exporting to File
File available for download


### Download Data from the server

In [25]:
remote.content('projects/demo/jupyter-remote')

,name,path,last_modified,created,content,format,mimetype,size,writable,type
0,lquant.log,projects/demo/jupyter-remote/lquant.log,2022-05-19T14:23:32.960801Z,2022-05-19T14:23:32.960801Z,None,None,None,182627,True,file
1,lquant-error.log,projects/demo/jupyter-remote/lquant-error.log,2022-03-23T18:38:17.828686Z,2022-03-23T18:38:17.828686Z,None,None,None,0,True,file
2,sp500.csv,projects/demo/jupyter-remote/sp500.csv,2022-05-19T14:23:33.000793Z,2022-05-19T14:23:33.000793Z,None,None,text/csv,35914,True,file
3,Remote Jupyter Demo.ipynb,projects/demo/jupyter-remote/Remote Jupyter De...,2022-05-19T14:12:57.919016Z,2022-05-19T14:12:57.919016Z,None,None,None,1379,True,notebook


In [115]:
data = remote.content('projects/demo/jupyter-remote/sp500.csv')

In [39]:
??remote.content

In [116]:
data

,Unnamed: 0,DATE,ID,PRCCD,IN_SP500
0,0,2022-01-31,1045.04,16.47,1.0
1,1,2022-01-31,1075.01,69.61,1.0
2,2,2022-01-31,1078.01,127.46,1.0
3,3,2022-01-31,1161.01,114.25,1.0
4,4,2022-01-31,1209.01,282.12,1.0
...,...,...,...,...,...
1009,1009,2022-02-28,188255.01,86.90,1.0
1010,1010,2022-02-28,189491.01,88.95,1.0
1011,1011,2022-02-28,260774.01,96.85,1.0
1012,1012,2022-02-28,294524.01,97.23,1.0


In [35]:
response = remote.execute('12263c47-20c1-4311-b93a-bf79911e8042', notebook.loc[2,'source'])

Fetching Data Done.
Exporting to File
File available for download


In [117]:
## Server Code
port = wq.port_upload_file('MYCDTEST','/mnt/ebs1/karora/projects/demo/datafiles/mydata.csv',
                                _global=False,pit_id=True,short_format=False)

NameError: name 'wq' is not defined

In [119]:
import pandas as pd
import json

def is_server_code(code):
    
    #print(type(code))
    if type(code) == list:
        if len(code) < 1:
            return False
        return code[0].startswith('## Server Code')
    
    if type(code) == str:
        return code.startsWith('## Server Code')
    
    return False

with open('RemoteJupyter.ipynb') as f:
    v = json.load(f)
    local_notebook = pd.DataFrame(v['cells'])
    local_notebook = local_notebook[local_notebook.cell_type == 'code']
    local_notebook = local_notebook[[is_server_code(x) for x in local_notebook.source]]


In [120]:
local_notebook

,cell_type,metadata,source,execution_count,outputs
19,code,{},"[## Server Code\n, port = wq.port_upload_file(...",NaN,[]
23,code,{},"[## Server Code\n, print(""Fetching Data...."")\...",103.0,"[{'name': 'stdout', 'output_type': 'stream', '..."


In [121]:
response = remote.execute('12263c47-20c1-4311-b93a-bf79911e8042', "".join(local_notebook.iloc[0].source))

In [122]:
response.is_error()

False

In [103]:
## Server Code
print("Fetching Data....")
r1 = wq.new_request().runFor('CIQMSCI_379947354').start('2022-01-31').to('2022-04-30').at('1me').a('TICKER').addInFlag()
data1 = wq.get_data(r1)
print("Fetching Data Done.")
df = data1.as_large_data_frame()
print("Exporting to File")
df.to_csv('acwi.csv')
print("File available for download")

Fetching Data....


NameError: name 'wq' is not defined

In [123]:
response = remote.execute('12263c47-20c1-4311-b93a-bf79911e8042', "".join(local_notebook.iloc[1].source))

Fetching Data....
Fetching Data Done.
Exporting to File
File available for download
